In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
import numpy as np
from tensorflow.keras.layers import LSTM
import pickle

# Load training data from text file
def load_training_data(file_path):
    texts = []
    labels = []
    label_map = {"Obtainable": 0, "Operable": 1, "Update": 2}  # Define label mapping
    with open(file_path, 'r') as file:
        for line in file:
            text, label = line.strip().split(',')
            texts.append(text)
            labels.append(label_map[label])  # Map label string to integer
    return texts, labels

# Sample dataset (for testing)
texts, labels = load_training_data('training_data.txt')

# Parameters
vocab_size = 5000
embedding_dim = 100
max_length = 20
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Tokenization and padding
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Preparing labels
labels = np.array(labels)

# Building the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the model
model.fit(padded_sequences, labels, epochs=10, verbose=2)

# Save the tokenizer for later use
tokenizer_path = "tokenizer.pickle"
with open(tokenizer_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the model in the native Keras format
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.export('raw-data')
model.save("raw-data/SaveState.keras")
#model.save('preproccessedModel.keras')

Epoch 1/10
2/2 - 5s - 2s/step - accuracy: 0.2632 - loss: 1.1034
Epoch 2/10
2/2 - 0s - 166ms/step - accuracy: 0.3509 - loss: 1.0983
Epoch 3/10
2/2 - 0s - 87ms/step - accuracy: 0.4386 - loss: 1.0959
Epoch 4/10
2/2 - 0s - 65ms/step - accuracy: 0.5614 - loss: 1.0929
Epoch 5/10
2/2 - 0s - 55ms/step - accuracy: 0.5789 - loss: 1.0922
Epoch 6/10
2/2 - 0s - 54ms/step - accuracy: 0.5614 - loss: 1.0891
Epoch 7/10
2/2 - 0s - 54ms/step - accuracy: 0.6140 - loss: 1.0769
Epoch 8/10
2/2 - 0s - 41ms/step - accuracy: 0.6140 - loss: 1.0526
Epoch 9/10
2/2 - 0s - 51ms/step - accuracy: 0.6667 - loss: 1.0102
Epoch 10/10
2/2 - 0s - 87ms/step - accuracy: 0.6667 - loss: 0.9124
INFO:tensorflow:Assets written to: raw-data\assets


INFO:tensorflow:Assets written to: raw-data\assets


Saved artifact at 'raw-data'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 20), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2681841139536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681841139152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681860031312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681860032272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681860032656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2681860033424: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Load the compiled Keras model
model = load_model('raw-data/SaveState.keras')

# Export the model to the SavedModel format
# model.save('tf_model')

# Convert the SavedModel to TFLite format with experimental flags
converter = tf.lite.TFLiteConverter.from_saved_model('raw-data')
converter.experimental_enable_resource_variables = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

# Perform the conversion
tflite_model = converter.convert()

# Save the TFLite model
with open('1stNeuralLink.tflite', 'wb') as f:
    f.write(tflite_model)

print("TFLite built, converted and saved successfully!")


h:\DevKit\projects\neuralnet\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


TFLite built, converted and saved successfully!
